In [1]:
import pandas as pd

# cc = pd.read_csv("Data/coaches_career.txt")
# cs = pd.read_csv("Data/coaches_season.txt")
# # draft = pd.read_csv("Data/draft.txt")
# pa = pd.read_csv("Data/player_allstar.txt")
# pp = pd.read_csv("Data/player_playoffs.txt")
# ppc = pd.read_csv("Data/player_playoffs_career.txt")
# prs = pd.read_csv("Data/player_regular_season.txt")
# prsc = pd.read_csv("Data/player_regular_season_career.txt")
# player = pd.read_csv("Data/players.txt")
# ts = pd.read_csv("Data/team_season.txt")
# team = pd.read_csv("Data/teams.txt")
# pbp = pd.read_csv("Kaggle data/play_by_play.csv", nrows=1000)

pd.set_option('display.max_rows', None)


In [2]:
def combinePlayerTeam(row, playerCol, teamCol):
    return [str(row[playerCol][i])+"_"+str(row[teamCol][i]) for i in range(0,len(row[teamCol]))]

def getPlayerInGame(pbp):
    df = pbp.groupby('game_id').agg({'player1_id': list, 'player1_team_abbreviation': list, 'player2_id': list, 'player2_team_abbreviation': list, 'player3_id': list, 'player3_team_abbreviation': list})
    df["combined"] = df.apply(lambda x: list(set(combinePlayerTeam(x, "player1_id", "player1_team_abbreviation") + combinePlayerTeam(x, "player2_id", "player2_team_abbreviation") +combinePlayerTeam(x, "player3_id", "player3_team_abbreviation"))), axis=1)
    df = df.explode("combined")
    df["player_id"] = df.apply(lambda x: x["combined"].split("_")[0], axis = 1)
    df["team_id"] = df.apply(lambda x: x["combined"].split("_")[1], axis = 1)

    return df[["player_id","team_id"]]

gameToPlayerChunks = []
count = 1
for df in pd.read_csv('Kaggle data/play_by_play.csv', chunksize=10000):
    if count % 100 == 0:
        chunk = pd.concat(gameToPlayerChunks)
        chunk.to_csv("Chunks/chunk_"+str(count/100)+".csv")
    count+=1
    gameToPlayerChunks.append(getPlayerInGame(df))
gameToPlayer = pd.concat(gameToPlayerChunks)

In [3]:
gameToPlayer.dropna(subset=["player_id"], inplace=True)
gameToPlayer = gameToPlayer[gameToPlayer['player_id'] != 0]
gameToPlayer.to_csv("gameToPlayer.csv")

In [98]:
gameToPlayer

In [4]:
pp

,ilkid,year,firstname,lastname,team,leag,gp,minutes,pts,dreb,...,stl,blk,turnover,pf,fga,fgm,fta,ftm,tpa,tpm
0,ARMSTPA01,1949,Paul,Armstrong,FTW,N,3,0,9,0,...,0,0,0,6,22,4,4,1,0,0
1,BARKECL01,1949,Cliff,Barker,INI,N,6,0,34,0,...,0,0,0,10,31,12,15,10,0,0
2,BARNHLE01,1949,Leo,Barnhorst,CH1,N,2,0,22,0,...,0,0,0,10,25,8,6,6,0,0
3,BEARDRA01,1949,Ralph,Beard,INI,N,5,0,66,0,...,0,0,0,11,70,22,28,22,0,0
4,BLACKCH01,1949,Charlie,Black,AND,N,8,0,57,0,...,0,0,0,38,61,18,29,21,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7538,WESTDE01,2004,Delonte,West,BOS,N,7,115,29,7,...,7,1,2,17,21,11,4,2,11,5
7539,WILKIDA02,2004,Damien,Wilkins,SEA,N,7,136,39,9,...,10,1,8,17,36,16,9,4,11,3
7540,WILLICO02,2004,Corliss,Williamson,SAC,N,5,40,26,2,...,1,2,4,11,16,6,18,14,1,0
7541,WILLIJA02,2004,Jason,Williams,MEM,N,4,114,68,8,...,6,0,8,7,53,28,2,2,21,10


In [9]:
prsc

,ilkid,firstname,lastname,leag,gp,minutes,pts,oreb,dreb,reb,...,stl,blk,turnover,pf,fga,fgm,fta,ftm,tpa,tpm
0,ABDELAL01,Alaa,Abdelnaby,N,256,3200,1465,283,563,846,...,71,69,247,484,1236,620,321,225,3,0
1,ABDULKA01,Kareem,Abdul-jabbar,N,1560,57446,38387,2975,9394,17440,...,1160,3189,2527,4657,28307,15837,9304,6712,18,1
2,ABDULMA01,Mahmo,Abdul-rauf,N,586,15633,8553,219,868,1087,...,487,46,963,1107,7943,3514,1161,1051,1339,474
3,ABDULTA01,Tariq,Abdul-wahad,N,236,4808,1830,275,501,776,...,184,82,309,485,1726,720,529,372,76,18
4,ABDURSH01,Shareef,Abdur-rahim,N,672,24862,13338,1498,3976,5474,...,718,556,1911,1845,10215,4789,4427,3614,477,146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3754,ZEVENPH01,Phil,Zevenbergen,N,8,58,30,4,9,13,...,3,1,4,12,27,15,2,0,0,0
3755,ZIDEKGE01,George,Zidek,N,135,1328,453,108,178,286,...,14,12,69,241,395,161,166,130,4,1
3756,ZOETJI01,Jim,Zoet,N,7,30,2,3,5,8,...,1,3,4,9,5,1,0,0,0,0
3757,ZOPFBI01,Bill,Zopf,N,53,398,118,0,0,46,...,0,0,0,34,135,49,36,20,0,0


In [35]:
cc.columns

Index(['coachid', 'firstname', 'lastname', 'season_win', 'season_loss',
       'playoff_win', 'playoff_loss'],
      dtype='object')

In [9]:
print(len(prsc.ilkid.unique()))
print(len(ppc.ilkid.unique()))

3572
1956


In [10]:
d={}
a = []
for _, row in ppc.iterrows():
  if row.ilkid not in d:
    d[row.ilkid] = row
  else:
    a.append(row)
    a.append(d[row.ilkid])
df = pd.DataFrame(a).sort_values('ilkid')
df[df['leag'] == 'N']

,ilkid,firstname,lastname,leag,gp,minutes,pts,dreb,oreb,reb,...,stl,blk,turnover,pf,fga,fgm,fta,ftm,tpa,tpm
8,ADAMSDO01,Don,Adams,N,22,566,156,0,0,116,...,0,0,0,74,165,63,44,30,0,0
47,ARDJI01,Jim,Ard,N,21,124,39,0,0,28,...,0,0,0,34,37,14,14,11,0,0
88,BARNHJO01,John,Barnhill,N,21,421,111,0,0,41,...,0,0,0,42,113,46,31,19,0,0
97,BARRYRI01,Rick,Barry,N,74,2723,1833,6,0,418,...,1,3,10,232,1688,719,448,392,12,3
99,BASSETI01,Tim,Bassett,N,5,36,8,0,1,3,...,0,0,1,11,7,3,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1990,WILLIJO01,John,Williams,N,27,749,283,78,41,125,...,24,18,39,78,246,110,91,59,11,4
2010,WILSOBO01,Bob,Wilson,N,10,93,44,0,0,11,...,0,0,0,10,41,17,12,10,0,0
2022,WISEWI01,Willie,Wise,N,6,106,45,0,0,28,...,0,0,0,16,41,14,25,17,0,0
2037,WORKMTO01,Tom,Workman,N,1,2,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [11]:
ts

,team,year,leag,o_fgm,o_fga,o_ftm,o_fta,o_oreb,o_dreb,o_reb,...,d_pf,d_stl,d_to,d_blk,d_3pm,d_3pa,d_pts,pace,won,lost
0,BOS,1946,N,1397,5133,811,1375,0,0,0,...,0,0,0,0,0,0,3900,0.000000,22,38
1,CH1,1946,N,1879,6309,939,1550,0,0,0,...,0,0,0,0,0,0,4471,0.000000,39,22
2,CL1,1946,N,1674,5699,903,1428,0,0,0,...,0,0,0,0,0,0,4308,0.000000,30,30
3,DE1,1946,N,1437,5843,923,1494,0,0,0,...,0,0,0,0,0,0,3918,0.000000,20,40
4,NYK,1946,N,1465,5255,951,1438,0,0,0,...,0,0,0,0,0,0,3840,0.000000,33,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182,SAS,2004,N,2923,6450,1535,2120,987,2489,3476,...,1891,594,1246,421,323,881,7248,91.455696,59,23
1183,SEA,2004,N,2882,6498,1683,2131,1041,2311,3352,...,1948,508,1127,390,470,1317,7925,91.244949,52,30
1184,TOR,2004,N,2952,6656,1626,2101,844,2444,3288,...,1795,507,1188,360,435,1222,8311,95.012611,33,49
1185,UTA,2004,N,2828,6301,1719,2272,1047,2243,3290,...,2057,646,1213,474,486,1297,7975,91.195145,26,56


In [92]:
ts

,team,year,leag,o_fgm,o_fga,o_ftm,o_fta,o_oreb,o_dreb,o_reb,o_asts,o_pf,o_stl,o_to,o_blk,o_3pm,o_3pa,o_pts,d_fgm,d_fga,d_ftm,d_fta,d_oreb,d_dreb,d_reb,d_asts,d_pf,d_stl,d_to,d_blk,d_3pm,d_3pa,d_pts,pace,won,lost
0,BOS,1946,N,1397,5133,811,1375,0,0,0,470,1202,0,0,0,0,0,3605,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3900,0.000000,22,38
1,CH1,1946,N,1879,6309,939,1550,0,0,0,436,1473,0,0,0,0,0,4697,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4471,0.000000,39,22
2,CL1,1946,N,1674,5699,903,1428,0,0,0,494,1246,0,0,0,0,0,4251,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4308,0.000000,30,30
3,DE1,1946,N,1437,5843,923,1494,0,0,0,482,1351,0,0,0,0,0,3797,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3918,0.000000,20,40
4,NYK,1946,N,1465,5255,951,1438,0,0,0,457,1218,0,0,0,0,0,3881,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3840,0.000000,33,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182,SAS,2004,N,2923,6450,1535,2120,987,2489,3476,1771,1716,613,1126,543,507,1395,7888,2712,6359,1501,1955,893,2403,3296,1385,1891,594,1246,421,323,881,7248,91.455696,59,23
1183,SEA,2004,N,2882,6498,1683,2131,1041,2311,3352,1487,1943,553,1113,338,666,1824,8113,2932,6382,1591,2124,930,2174,3104,1698,1948,508,1127,390,470,1317,7925,91.244949,52,30
1184,TOR,2004,N,2952,6656,1626,2101,844,2444,3288,1670,1875,621,1087,317,648,1681,8178,3133,6708,1610,2165,991,2720,3711,1814,1795,507,1188,360,435,1222,8311,95.012611,33,49
1185,UTA,2004,N,2828,6301,1719,2272,1047,2243,3290,1826,2189,541,1292,374,250,762,7625,2731,5965,2027,2653,859,2230,3089,1592,2057,646,1213,474,486,1297,7975,91.195145,26,56


In [100]:
import numpy as np
currTeam = "LAL"
y = ts[(ts["team"] == currTeam) & (ts["year"] < 2004) & (ts["year"] > 1950) & (ts["leag"] == 'N')]
print("won:"+str(np.sum(y["won"])))
print("lost:"+str(np.sum(y["lost"])))

won:2270
lost:1292


In [101]:
game = pd.read_csv("Kaggle Data/game.csv")
game["year"] = game.apply(lambda x: int(x["game_date"][0:4]), axis = 1)
game = game[(game["year"] < 2004) & (game["year"] > 1950)]

In [102]:
x = game[((game["team_abbreviation_home"] == currTeam) & (game["wl_home"] == "W")) | ((game["team_abbreviation_away"] == currTeam) & (game["wl_away"] == "W"))]
z = game[((game["team_abbreviation_home"] == currTeam) & (game["wl_home"] == "L")) | ((game["team_abbreviation_away"] == currTeam) & (game["wl_away"] == "L"))]

print("w:"+str(x.shape[0]))
print("z:"+str(z.shape[0]))


w:2216
z:1222


In [99]:
game

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,fga_home,fg_pct_home,fg3m_home,fg3a_home,fg3_pct_home,ftm_home,fta_home,ft_pct_home,oreb_home,dreb_home,reb_home,ast_home,stl_home,blk_home,tov_home,pf_home,pts_home,plus_minus_home,video_available_home,team_id_away,team_abbreviation_away,team_name_away,matchup_away,wl_away,fgm_away,fga_away,fg_pct_away,fg3m_away,fg3a_away,fg3_pct_away,ftm_away,fta_away,ft_pct_away,oreb_away,dreb_away,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,video_available_away,season_type,year
1696,21950,1610612765,FTW,Ft. Wayne Zollner Pistons,25000165,1951-01-01 00:00:00,FTW vs. MNL,W,0,27.0,NaN,NaN,NaN,NaN,NaN,29.0,37.0,0.784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.0,13,0,1610612747,MNL,Minneapolis Lakers,MNL @ FTW,L,22.0,NaN,NaN,NaN,NaN,NaN,23.0,30.0,0.767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,-13,0,Regular Season,1951
1697,21950,1610612755,SYR,Syracuse Nationals,25000162,1951-01-01 00:00:00,SYR vs. BOS,W,0,34.0,NaN,NaN,NaN,NaN,NaN,38.0,48.0,0.792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.0,17,0,1610612738,BOS,Boston Celtics,BOS @ SYR,L,32.0,NaN,NaN,NaN,NaN,NaN,25.0,40.0,0.625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.0,-17,0,Regular Season,1951
1698,21950,1610612737,TCB,Tri-Cities Blackhawks,25000161,1951-01-01 00:00:00,TCB vs. PHW,W,0,38.0,NaN,NaN,NaN,NaN,NaN,33.0,48.0,0.688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109.0,17,0,1610612744,PHW,Philadelphia Warriors,PHW @ TCB,L,27.0,NaN,NaN,NaN,NaN,NaN,38.0,48.0,0.792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92.0,-17,0,Regular Season,1951
1699,21950,1610610024,BAL,Baltimore Bullets,25000164,1951-01-01 00:00:00,BAL vs. WAS,W,0,24.0,NaN,NaN,NaN,NaN,NaN,35.0,45.0,0.778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.0,12,0,1610610036,WAS,Washington Capitols,WAS @ BAL,L,26.0,NaN,NaN,NaN,NaN,NaN,19.0,30.0,0.633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.0,-12,0,Regular Season,1951
1700,21950,1610612758,ROC,Rochester Royals,25000163,1951-01-01 00:00:00,ROC vs. NYK,W,0,31.0,NaN,NaN,NaN,NaN,NaN,29.0,36.0,0.806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.0,3,0,1610612752,NYK,New York Knicks,NYK @ ROC,L,31.0,NaN,NaN,NaN,NaN,NaN,26.0,33.0,0.788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,-3,0,Regular Season,1951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40217,22003,1610612745,HOU,Houston Rockets,20300450,2003-12-31 00:00:00,HOU vs. PHI,W,240,31.0,77.0,0.403,10.0,24.0,0.417,8.0,13.0,0.615,16.0,34.0,50.0,18.0,8.0,2.0,19.0,23.0,80.0,8,0,1610612755,PHI,Philadelphia 76ers,PHI @ HOU,L,26.0,75.0,0.347,6.0,12.0,0.500,14.0,23.0,0.609,10.0,31.0,41.0,17.0,5.0,2.0,15.0,17.0,72.0,-8,0,Regular Season,2003
40218,22003,1610612765,DET,Detroit Pistons,20300447,2003-12-31 00:00:00,DET vs. POR,W,240,32.0,77.0,0.416,4.0,7.0,0.571,10.0,12.0,0.833,11.0,30.0,41.0,22.0,8.0,9.0,10.0,12.0,78.0,7,0,1610612757,POR,Portland Trail Blazers,POR @ DET,L,28.0,70.0,0.400,3.0,11.0,0.273,12.0,16.0,0.750,9.0,27.0,36.0,18.0,5.0,6.0,11.0,12.0,71.0,-7,0,Regular Season,2003
40219,22003,1610612746,LAC,Los Angeles Clippers,20300449,2003-12-31 00:00:00,LAC vs. DEN,W,240,46.0,89.0,0.517,6.0,11.0,0.545,22.0,31.0,0.710,15.0,31.0,46.0,29.0,9.0,5.0,11.0,23.0,120.0,16,0,1610612743,DEN,Denver Nuggets,DEN @ LAC,L,40.0,81.0,0.494,5.0,15.0,0.333,19.0,23.0,0.826,4.0,26.0,30.0,23.0,7.0,6.0,15.0,29.0,104.0,-16,0,Regular Season,2003
40220,22003,1610612751,NJN,New Jersey Nets,20300448,2003-12-31 00:00:00,NJN vs. GSW,W,240,34.0,78.0,0.436,7.0,16.0,0.438,13.0,15.0,0.867,9.0,27.0,36.0,23.0,12.0,2.0,11.0,13.0,88.0,18,0,1610612744,GSW,Golden State Warriors,GSW @ NJN,L,28.0,70.0,0.400,3.0,16.0,0.188,11.0,17.0,0.647,9.0,31.0,40.0,15.0,8.0,4.0,16.0,16.0,70.0,-18,0,Regular Season,2003
